In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# 그래프에서 한글 깨짐 방지
import platform
if platform.system() == "Darwin":          # MAC인 경우
    plt.rc('font', family ="AppleGothic")
else:
    plt.rc('font',family='Malgun Gothic')

#멧플로립에서 음수를 표시
plt.rc('axes',unicode_minus=False)

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('../외감기업/raw_data/t-1_raw_외감.csv',encoding='euc-kr')

In [3]:
df['자산총계(천원)'] = df['자산총계(천원)'] * 1000
df['log자산총계'] = np.log(df['자산총계(천원)'])
df['잉여현금흐름(FCF)']=df['OCF(천원)']-df['ICF(천원)']
df['전체현금흐름(천원)']=df['OCF(천원)']+df['ICF(천원)']+df['FCF(천원)']

df['총자산대비잉여현금흐름'] = df['잉여현금흐름(FCF)']/df['자산총계(천원)']
df['총자산대비현금흐름'] = df['전체현금흐름(천원)'] / df['자산총계(천원)']
df=df[['회사명', '거래소코드','유동자산회전률', '총자산대비잉여현금흐름','자기자본구성비율', 'log자산총계','자기자본회전률', '순운전자본회전률', '자기자본증가율', '총자본증가율', '총자산대비현금흐름', '총자본투자효율','t-1감사의견코드']]

---
- 결측치, 무한대값 삭제

In [4]:
df.isna().sum()
df.dropna(axis=0,inplace=True)

In [5]:
df.isin([np.inf,-np.inf]).sum()
df = df.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 대체
df = df.dropna()  # NaN이 있는 행 삭제

---

In [12]:
# std, Mean·Median
df.describe()

,거래소코드,유동자산회전률,총자산대비잉여현금흐름,자기자본구성비율,log자산총계,자기자본회전률,순운전자본회전률,자기자본증가율,총자본증가율,총자산대비현금흐름,총자본투자효율,t-1감사의견코드
count,183883.000000,183883.000000,183883.000000,183883.000000,183883.000000,183883.00000,183883.000000,183883.000000,183883.000000,1.838830e+05,183883.000000,183883.000000
mean,57339.266436,3.030888,0.000189,38.352607,24.116252,11.47110,1.361660,12.616087,11.329977,1.058298e-05,21.094063,0.071295
std,25339.702565,3.472119,0.036187,27.772928,1.040771,1395.00394,1.545144,56.172506,1198.074219,7.724358e-05,1993.230970,0.257318
min,10000.000000,0.000000,-0.013627,-22.870000,14.021711,-0.04000,0.000000,-80.480000,-100.000000,-3.743830e-04,-680430.510000,0.000000
25%,41619.000000,1.010000,-0.000003,15.000000,23.411666,0.50000,0.420000,0.000000,-2.230000,-1.165009e-05,4.630000,0.000000
50%,59356.000000,2.130000,0.000078,35.150000,23.892937,1.83000,1.030000,1.280000,0.000000,9.913257e-07,11.520000,0.000000
75%,76333.000000,3.750000,0.000209,59.200000,24.611120,4.45000,1.740000,11.880000,10.210000,2.683828e-05,19.910000,0.000000
max,107278.000000,33.240000,15.517038,106.700000,31.807494,580354.99000,48.550000,902.800000,513178.820000,4.549450e-04,339685.680000,1.000000


In [9]:
import numpy as np

# 컬럼 이름
column_name = '자기자본증가율'

# 하위 1%, 상위 1% 값을 계산
lower_bound = df[column_name].quantile(0.005)
upper_bound = df[column_name].quantile(0.995)

# 1% 미만 및 1% 초과 데이터 선택 후 삭제
df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

In [ ]:
import numpy as np

# 컬럼 이름
column_name = '유동자산회전률'

# 하위 1%, 상위 1% 값을 계산
lower_bound = df[column_name].quantile(0.005)
upper_bound = df[column_name].quantile(0.995)

# 1% 미만 및 1% 초과 데이터 선택 후 삭제
df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

In [11]:
import numpy as np

# 컬럼 이름
column_name = '총자산대비현금흐름'

# 하위 1%, 상위 1% 값을 계산
lower_bound = df[column_name].quantile(0.005)
upper_bound = df[column_name].quantile(0.995)

# 1% 미만 및 1% 초과 데이터 선택 후 삭제
df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

---

In [13]:
scaler=MinMaxScaler()
df.iloc[:,2:12]=scaler.fit_transform(df.iloc[:,2:12])

In [14]:
# dp-Score의 계수
coefficients = {
    '유동자산회전률':np.exp(0.47765086),                    # 높아야 좋다
    '총자산대비잉여현금흐름':np.exp(0.53785436),            # 높아야 좋다
    '자기자본구성비율':np.exp(0.49360789),                  # 높아야 좋다
    'log자산총계':np.exp(0.94238478),                      # 높아야 좋다
    '자기자본회전률':np.exp(-0.08646556),                   # 높아야 좋다
    '순운전자본회전률':np.exp(-0.98107367),                 # 높아야 좋다
    '자기자본증가율':np.exp(0.19544705),                    # 높아야 좋다
    '총자본증가율':np.exp(0.44094258),                      # 높아야 좋다
    '총자산대비현금흐름':np.exp(-0.37036301),               # 높아야 좋다
    '총자본투자효율':np.exp(0.26458857),                    # 높아야 좋다

    'constant': 0.90495308
}


# O-Score 계산
df['dp_score'] = coefficients['constant']
df['dp_score'] += coefficients['유동자산회전률'] * df['유동자산회전률']
df['dp_score'] += coefficients['총자산대비잉여현금흐름'] * df['총자산대비잉여현금흐름']
df['dp_score'] += coefficients['자기자본구성비율'] * df['자기자본구성비율']
df['dp_score'] += coefficients['log자산총계'] * df['log자산총계']
df['dp_score'] += coefficients['자기자본회전률'] * df['자기자본회전률']
df['dp_score'] += coefficients['순운전자본회전률'] * df['순운전자본회전률']
df['dp_score'] += coefficients['자기자본증가율'] * df['자기자본증가율']
df['dp_score'] += coefficients['총자본증가율'] * df['총자본증가율']
df['dp_score'] += coefficients['총자산대비현금흐름'] * df['총자산대비현금흐름']
df['dp_score'] += coefficients['총자본투자효율'] * df['총자본투자효율']

# 로지스틱 함수를 이용해 부도 확률 계산
# df_등급화['default_probability'] = 1 / (1 + np.exp(-df_등급화['dp_score']))

In [15]:
df.shape

(183883, 14)

In [16]:
df.describe()

,거래소코드,유동자산회전률,총자산대비잉여현금흐름,자기자본구성비율,log자산총계,자기자본회전률,순운전자본회전률,자기자본증가율,총자본증가율,총자산대비현금흐름,총자본투자효율,t-1감사의견코드,dp_score
count,183883.000000,183883.000000,183883.000000,183883.000000,183883.000000,1.838830e+05,183883.000000,183883.000000,183883.000000,183883.000000,183883.000000,183883.000000,183883.000000
mean,57339.266436,0.091182,0.000890,0.472506,0.567562,1.983458e-05,0.028047,0.094679,0.000217,0.464190,0.667033,0.071295,4.599549
std,25339.702565,0.104456,0.002330,0.214347,0.058517,2.403708e-03,0.031826,0.057128,0.002334,0.093140,0.001954,0.257318,0.436639
min,10000.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.731876
25%,41619.000000,0.030385,0.000877,0.292274,0.527947,9.304649e-07,0.008651,0.081849,0.000190,0.437382,0.667017,0.000000,4.271484
50%,59356.000000,0.064079,0.000882,0.447789,0.555007,3.222166e-06,0.021215,0.083150,0.000195,0.452625,0.667024,0.000000,4.592944
75%,76333.000000,0.112816,0.000891,0.633403,0.595386,7.736644e-06,0.035839,0.093931,0.000215,0.483791,0.667032,0.000000,4.919503
max,107278.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.471289


In [17]:
df[df['t-1감사의견코드']==1]['dp_score'].mean()

4.4145129979814115

- AA	4.859760
- A	4.704991
- BBB	4.456432
- BB	4.253703
- B	4.105671
- C	3.954838